<a href="https://colab.research.google.com/github/aSiri13/Routing-Agent/blob/main/ReACT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup Data + Build Query Engine Tools

#Setup Base ReAct Agent (gpt-3.5-turbo)

This (not trained) ReAct agent can sometimes enter the incorrect reasoning loop to answer the question.

In [ ]:
!pip install llama_index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.9 MB/s eta 0:00:00


In [ ]:
from llama_index.core.agent import ReActAgent


In [ ]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.tools.google import OpenCV
from llama_index.core.tools import FunctionTool

tool_spec = OpenCV()

# Create a custom tool.
def location() -> string:
    return ""

function_tool = FunctionTool.from_defaults(fn=location)

tools = tool_spec.to_tool_list() + [function_tool]
agent = OpenAIAgent.from_tools(tools, verbose=True)

# use agent
agent.chat(
    "Can you tell me where the apple is"
)

ModuleNotFoundError: No module named 'llama_index.tools'

setup llm and agent using base llm

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-0613")
# llm = OpenAI(model="gpt-4-0613")
base_agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

NameError: name 'OpenAI' is not defined

Bad response below

In [ ]:
response = base_agent.chat(
    "User query goes here"
)

print(str(response))

NameError: name 'base_agent' is not defined

bad response (x2)

In [ ]:
response = base_agent.chat(
    "User query goes here"
)
print(str(response))

Thought: I need to use a tool to help me answer the question.
Action: march_2022
Action Input: {'input': 'risk factors'}
Observation: The company faces various risk factors that could have an adverse effect on its business, financial condition, and results of operations. These risk factors include the negative impact of economic, social, weather, and regulatory conditions on its operations, the potential failure or inferior performance of autonomous vehicle technologies, the need to retain and attract high-quality personnel, the risk of security breaches and data privacy breaches, the potential harm from cyberattacks, the impact of climate change risks, the reliance on third parties for distribution and software, the need for additional capital, the risks associated with identifying and integrating suitable businesses, the limitations on providing services in certain jurisdictions, the legal and regulatory risks, the risks related to payment and financial services regulations, the risk

#Generate Training/Eval Questions

- Generate a synthetic dataset of questions to ask (later comes from visual Q&A dataset).
- generate an initial set of questions over a “base” document (the March 2022 10Q)
- use an LLM to generate variations of that question that can apply across multiple quarters (later apply across many types of sorroundings).

Aim: stress-test the LLM reasoning capabilities.

In [ ]:
from llama_index.core.llama_dataset.generator import RagDatasetGenerator

In [ ]:
base_question_gen_query = (
    "You are a Teacher/ Professor. Your task is to setup a quiz/examination."
    " Using the provided data to generate questions."
)

dataset_generator = RagDatasetGenerator.from_documents(
    #march_docs, - replace with new data docs
    question_gen_query=base_question_gen_query,
    llm=llm_35
)

Need to fix dataset generator - these questions will be from the CV & location Q&A dataset

In [ ]:
questions = [""]

Variations of each question

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core import PromptTemplate


vary_question_tmpl = """\
You are a visual assistant. Given a question over a {data docs}, your goal
is to generate up to {num_vary} variations of that question that might span multiple 10Q's.

This can include compare/contrasting different 10Qs, replacing the current quarter with
another quarter, or generating questions that can only be answered over multiple quarters (be creative!)

You are given a valid set of 10Q filings. Please only generate question variations that can be
answered in that set.

For example:
Base Question: What color is the apple?
Valid 10Qs: [OpenCV, Knowledge Graph, Location]
Question Variations:
What color is the apple that is on the table?
Can you compare/contrast the apple on the table to apples from grocery stores?

Now let's give it a shot!

Base Question: {base_question}
Valid 10Qs: {valid_10qs}
Question Variations:
"""


def gen_question_variations(base_questions, num_vary=3):
    """Generate question variations."""

    VALID_10Q_STR = "[OpenCV, Knowledge Graph, Location]"

    llm = OpenAI(model="gpt-4")
    prompt_tmpl = PromptTemplate(vary_question_tmpl)

    new_questions = []
    for idx, question in enumerate(base_questions):
        new_questions.append(question)
        response = llm.complete(
            prompt_tmpl.format(
                num_vary=num_vary,
                base_question=question,
                valid_10qs=VALID_10Q_STR,
            )
        )
        # parse into newlines
        raw_lines = str(response).split("\n")
        cur_new_questions = [l for l in raw_lines if l != ""]
        print(f"[{idx}] Original Question: {question}")
        print(f"[{idx}] Generated Question Variations: {cur_new_questions}")
        new_questions.extend(cur_new_questions)

    return new_questions


def save_questions(questions, path):
    with open(path, "w") as f:
        for question in questions:
            f.write(question + "\n")


def load_questions(path):
    questions = []
    with open(path, "r") as f:
        for line in f:
            questions.append(line.strip())
    return questions

In [ ]:
new_questions = gen_question_variations(questions)

NameError: name 'gen_question_variations' is not defined

In [ ]:
len(new_questions)

80

60 training questions, 20 evaluating questions

In [ ]:
train_questions, eval_questions = new_questions[:60], new_questions[60:]

In [ ]:
save_questions(train_questions, "train_questions_10q.txt")
save_questions(eval_questions, "eval_questions_10q.txt")

In [ ]:
train_questions = load_questions("train_questions_10q.txt")
eval_questions = load_questions("eval_questions_10q.txt")

#Log Input/Output Pairs

- run the train questions through a ReAct agent (GPT-4) to collect prompt outputs
- Every prompt call to the LLM is logged as an input/output pair.
- OpenAIFineTuningHandler automatically collects prompt input/outputs when agent queries are run.
- This dataset can then be saved, in a dataset format .jsonl to directly feed to the OpenAI Finetuning endpoints to finetune 3.5 on 4 questions

create finetuning_handler + callback_manager
Also set context window length

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.finetuning.callbacks import OpenAIFineTuningHandler
from llama_index.core.callbacks import CallbackManager
from llama_index.core.agent import ReActAgent

finetuning_handler = OpenAIFineTuningHandler()
callback_manager = CallbackManager([finetuning_handler])

from llama_index.core import Settings

# limit the context window artifically to test refine process
Settings.context_window = 2048

Setup gpt-4 ReAct Agent

In [ ]:
llm = OpenAI(model="gpt-4-0613")
gpt4_agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    callback_manager=callback_manager,
    verbose=True,
)

In [ ]:
for idx, question in enumerate(train_questions):
    print(f"[{idx}] Question: {question}")
    response = gpt4_agent.query(question)
    print(f"[{idx}] Agent Response: {str(response)}")

Save events

In [ ]:
finetuning_handler.save_finetuning_events("finetuning_events_10q.jsonl")

Wrote 179 examples to finetuning_events_10q.jsonl


#Create OpenAIFinetuneEngine
the finetune engine will launch a finetuning job, and returning an LLM model that you can directly plugin to the rest of LlamaIndex workflows.

If finetuning the same model even further - list job id below

In [ ]:
from llama_index.finetuning import OpenAIFinetuneEngine

finetune_engine = OpenAIFinetuneEngine(
    "gpt-3.5-turbo",
    "finetuning_events_10q.jsonl",
    #start_job_id="<start-job-id>"  # if you have an existing job, can specify id here
)

finetune

In [ ]:
finetune_engine.finetune()

Num examples: 179
First example:
{'role': 'system', 'content': '\nYou are designed to help with a variety of tasks, from answering questions     to providing summaries to other types of analyses.\n\n## Tools\nYou have access to a wide variety of tools. You are responsible for using\nthe tools in any sequence you deem appropriate to complete the task at hand.\nThis may require breaking the task into subtasks and using different tools\nto complete each subtask.\n\nYou have access to the following tools:\n> Tool Name: march_2022\nTool Description: Provides information about Uber quarterly financials ending March 2022\nTool Args: {"type": "object", "properties": {"input": {"title": "Input", "type": "string"}}, "required": ["input"]}\n\n> Tool Name: june_2022\nTool Description: Provides information about Uber quarterly financials ending June 2022\nTool Args: {"type": "object", "properties": {"input": {"title": "Input", "type": "string"}}, "required": ["input"]}\n\n> Tool Name: sept_2022\nTo

get finetuning job id

In [ ]:
finetune_engine.get_current_job()

FineTuningJob(id='ftjob-qiGoHWTJZfc6iun19unXc2pV', created_at=1709675066, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-by7b0ByCCb8VyGQAQud5kEuU', result_files=[], status='queued', trained_tokens=None, training_file='file-Ii4qXOwTGc4VMrScvv0iZacL', validation_file=None, user_provided_suffix=None)

get finetuned llm - see status on OAI dev portal (must be successful)

In [ ]:
ft_llm = finetune_engine.get_finetuned_model(temperature=0.3)

#Compare Finetuned Agent vs. Base Agent
- run some sample queries from the evaluation dataset over both our finetuned agent as well as the base agent.
- qualitatively look at their abilities to perform chain of thought prompting in order to arrive at the right answer.

Setup FT agent using ft model

In [ ]:
ft_agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=ft_llm,
    callback_manager=callback_manager,
    verbose=True,
)

retrieve evaluation questions

In [ ]:
eval_questions = ["Is the teacher standing in front of the classroom while giving a lecture or conducting an activity?",
  "How does the classroom arrangement affect the teacher's ability to interact with students at their desks?",
  "Is the student writing an answer on the whiteboard as part of a group activity or individual response?",
  "Are the books on the teacher's desk related to the subject currently being taught or for future lessons?",
  "Is the projector used more frequently for presentations or for showing educational videos?",
  "How do the dynamics of students working in groups compare to individual work in terms of engagement and productivity?",
  "Can the student raising their hand to ask a question indicate a high level of classroom participation?",
  "Is the teacher using the map on the wall for a geography lesson or historical context?",
  "Do open windows in the classroom contribute to a better learning environment by improving air quality?",
  "Is the student using their laptop for research, writing, or browsing unrelated content?",
  "Are the pencils and pens organized by color or type in the holder?",
  "Is the teacher reading from a textbook as the primary source of information or supplementing with external materials?",
  "Are students more focused when watching an educational video compared to a traditional lecture?",
  "Is the clock in the classroom an effective tool for time management for both students and the teacher?",
  "How does the stress level of students taking a test compare with their usual classroom work?",
  "Is the teacher checking homework assignments for completeness or understanding of the material?",
  "Are the students' drawings in their sketchbooks related to the current curriculum or personal projects?",
  "Is the student using a tablet for educational apps, reading, or entertainment?",
  "How effective are audio lessons through headphones in maintaining students' concentration compared to classroom discussions?",
  "Is the teacher explaining a complex math problem on the projector screen more effective than traditional board work?",
  "Is the public library serving more as a study space or a resource center for the community?",
  "Can the proximity to a coffee shop enhance the appeal of a location for students or professionals?",
  "Does the presence of a lake in the nearest park increase its popularity for recreational activities?",
  "How do Italian restaurants in the neighborhood compare in terms of authenticity and price?",
  "Is the accuracy of the weather forecast impacting planning for outdoor activities in the area?",
  "Which is a bigger tourist attraction, the museum or the cinema, based on their proximity?",
  "Do grocery stores offering organic produce attract a more health-conscious clientele?",
  "How does the presence of a pet-friendly café within a 1-mile radius affect the popularity of the area?",
  "Are public transportation options adequate for commuters to the city center during peak hours?",
  "Does the region's recognition for historical landmarks contribute to its cultural significance?",
  "How reliable are gym opening hours for early morning or late-night fitness enthusiasts?",
  "Are flu vaccinations at the closest pharmacy drawing a larger crowd during flu season?",
  "Does being designated as a nature reserve increase the ecological awareness among visitors?",
  "How does the availability of a nearby bicycle rental service encourage eco-friendly tourism?",
  "Are lifeguards at the nearest beach adequately trained for emergency situations?",
  "How does the high population density of a zone affect the quality of life for its residents?",
  "Is the daily air quality index a significant concern for people with respiratory issues in the area?",
  "How do vegan restaurants within a 10-minute drive cater to the diverse dietary preferences of the community?",
  "Is the direct train service to the airport reliable for timely travel?",
  "Does the presence of a swimming pool at the nearest hotel enhance its appeal to tourists?",
  "How does the teacher's posture change when standing in front of different sections of the classroom?",
  "Can the students at their desks be grouped by their level of attentiveness?",
  "Which student is the most active in writing answers on the whiteboard, and how does their activity compare to the rest?",
  "Are the books on the teacher's desk more related to the current subject being taught or for future lessons?",
  "What is the topic of the presentation being displayed by the projector, and how does it relate to the students' current curriculum?",
  "How do the dynamics of student groups vary across different classroom activities?",
  "What is the frequency of students raising their hands to ask questions during a typical class session?",
  "Can the teacher's pointing to the map on the wall be correlated with the geographic focus of the current lesson?",
  "What is the impact of open windows in the classroom on the students' attentiveness and classroom temperature?",
  "Is the student typing on a laptop more engaged in academic activities or non-educational browsing?",
  "How does the organization of pencils and pens in the holder reflect the overall tidiness of the classroom?",
  "What subject is the teacher reading from the textbook, and how does it align with the students' interest levels?",
  "Are students more engaged when watching an educational video or during interactive classroom activities?",
  "How does the classroom's atmosphere change when the clock indicates it's time for recess?",
  "What strategies do students use to concentrate while taking a test in a noisy classroom?",
  "In what ways does the teacher provide feedback while checking homework assignments, and how is it received by the students?",
  "How do the themes of students' drawings in their sketchbooks relate to the subjects being taught in class?",
  "What content is the student primarily accessing on the tablet, educational or entertainment?",
  "How does listening to an audio lesson through headphones affect students' comprehension compared to traditional teaching methods?",
  "What are the most common errors made by students when solving the math problem explained on the projector screen?",
  "What distinct characteristics set a public library apart from a school library in terms of ambiance and resources?",
  "What are the most popular destinations within walking distance of the nearest coffee shop, and why?",
  "Does the presence of a lake in the nearest park increase its popularity among local residents?",
  "How do Italian restaurants in the current neighborhood compare in terms of authenticity and price?",
  "What are the implications of today's weather forecast on outdoor activities in the area?",
  "Which is a more popular local attraction, the museum or the cinema, and what factors contribute to its popularity?",
  "What variety of organic produce is offered by the closest grocery store, and how does it compare to non-organic options?",
  "What amenities do pet-friendly cafés within a 1-mile radius offer to attract pet owners?",
  "How reliable and timely are the public transportation services from this location to the city center?",
  "What historical landmarks in the region are most visited, and what stories do they tell?",
  "What are the peak hours for the nearest gym, and how crowded does it get?",
  "Are there any other health services provided by the closest pharmacy besides flu vaccinations?",
  "What flora and fauna can be found in the area designated as a nature reserve?",
  "How does the availability of bicycle rental services impact the local culture and transportation habits?",
  "What safety measures are in place at the nearest beach, and how do they affect visitors' experiences?",
  "How does the high population density in the zone impact the local community and infrastructure?",
  "What actions are being taken in the area to improve the air quality index to a healthy level?",
  "What unique offerings do vegan restaurants within a 10-minute drive provide to attract customers?",
  "What are the advantages and disadvantages of using the direct train service from the current location to the airport?",
  "What additional amenities does the nearest hotel offer to enhance guests' stays beyond having a swimming pool?"]
with open("eval_questions_10q.txt", "r") as f:
    for line in f:
        eval_questions.append(line.strip())

In [ ]:
qidx = 0
print(eval_questions[qidx])

What is the total fair value of Uber's financial assets as of March 31, 2022?


Get response from base agent
base agent = normal gpt-3.5

In [ ]:
base_response = base_agent.query(eval_questions[qidx])
print(str(base_response))

get response from ft agent
ft agent = finetuned gpt 3.5 on answers from gpt 4.0

In [ ]:
ft_response = ft_agent.query(eval_questions[qidx])
print(str(ft_response))

#TL;DR
- finetuned model does much better than the base model in terms of reasoning about the current sequence of steps @same costs.
- It passes more detailed answers to the downstream tools
- Is more capable of refining its approach when initial queries don’t work.